# Общие эпитопы ковида и человека
### Ищем регионы шириной 9 и 14 а.к., сходство минимум 90%
***

# [18.09]

In [56]:
peplen = 10

In [57]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

batch = 20
peps = []
with open("wuhan_hu_1.faa", "r") as handle:
    q = 1
    for record in SeqIO.parse(handle, "fasta"):
        for i in range(1, len(record.seq)-peplen):
            seq_name = record.id + "_" + str(i)
            seq = Seq(str(record.seq[i:i+peplen]))
            sr = SeqRecord(seq, seq_name, '', '')
            peps.append(sr)
            if(len(peps)>batch):
                fname="peptides/batch_"+str(q)+".fasta"
                with open(fname, "w") as output_handle:
                    SeqIO.write(peps, output_handle, "fasta")
                    q = q + 1
                    peps = []

In [ ]:
!blastp -db uniprot_human -query peptides/batch_84.fasta -word_size 3 -evalue 1000 -outfmt '6 std nident salltitles'

In [ ]:
!snakemake --cores 120 -rk > slog 2>&1

In [59]:
!mkdir -p result
!cat blast/*.out > result/blast.out 

In [62]:
print(peplen)
import pandas as pd
blast = pd.read_csv("result/blast.out", sep="\t", header = None)
blast.columns=["qseqid", 'sseqid', 'pident', 'length', 'mismatch', 'gapopen', 'qstart', 'qend', 'sstart', 'send', 'evalue', 'bitscore', 'nident','sname']
blast.head()
    

10


,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,nident,sname
0,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|B3KUB4|B3KUB4_HUMAN,75.000,8,2,0,1,8,139,146,47.0,18.5,6,tr|B3KUB4|B3KUB4_HUMAN Carbonic anhydrase OS=H...
1,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A096LP60|A0A096LP60_HUMAN,85.714,7,1,0,1,7,75,81,65.0,18.1,6,tr|A0A096LP60|A0A096LP60_HUMAN AP2-associated ...
2,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A096LP25|A0A096LP25_HUMAN,75.000,8,2,0,1,8,81,88,68.0,18.1,6,tr|A0A096LP25|A0A096LP25_HUMAN AP2-associated ...
3,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A096LNZ0|A0A096LNZ0_HUMAN,75.000,8,2,0,1,8,81,88,73.0,18.1,6,tr|A0A096LNZ0|A0A096LNZ0_HUMAN AP2-associated ...
4,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A0U1RRB6|A0A0U1RRB6_HUMAN,75.000,8,2,0,3,10,715,722,82.0,17.7,6,tr|A0A0U1RRB6|A0A0U1RRB6_HUMAN Exocyst complex...


In [67]:
blast_ok = blast[blast["nident"]==(peplen-1)]
blast_ok.to_csv("result/blast_filtered_9.out", sep="\t")
blast_ok.head()


,qseqid,sseqid,pident,length,mismatch,gapopen,qstart,qend,sstart,send,evalue,bitscore,nident,sname
849,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A0A087X1X6|A0A087X1X6_HUMAN,90.0,10,0,1,1,10,322,330,93.0,17.7,9,tr|A0A087X1X6|A0A087X1X6_HUMAN Forkhead box pr...
1000,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|B3KUL3|B3KUL3_HUMAN,50.0,18,1,1,1,10,85,102,82.0,17.7,9,tr|B3KUL3|B3KUL3_HUMAN Ferredoxin-fold anticod...
1001,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|F6VJS9|F6VJS9_HUMAN,50.0,18,1,1,1,10,9,26,83.0,17.7,9,tr|F6VJS9|F6VJS9_HUMAN Uncharacterized protein...
4647,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|A6NEM2|A6NEM2_HUMAN,90.0,10,1,0,1,10,839,848,4.5,21.6,9,tr|A6NEM2|A6NEM2_HUMAN Host cell factor 1 OS=H...
9907,wuhan_hu_1|1|orf1ab_polyprotein|MN908947|266|2...,tr|H0YJ08|H0YJ08_HUMAN,90.0,10,1,0,1,10,47,56,95.0,17.7,9,tr|H0YJ08|H0YJ08_HUMAN AP-1 complex subunit ga...


In [68]:
!git add result/blast_filtered_9.out
!git commit -am "add 9"
!git push

[l7 29487f7] add 9
 4 files changed, 52685 insertions(+), 11762 deletions(-)
 rewrite result/blast_filtered_7.out (99%)
 create mode 100644 result/blast_filtered_9.out
fatal: The upstream branch of your current branch does not match
the name of your current branch.  To push to the upstream branch
on the remote, use

    git push origin HEAD:master

To push to the branch of the same name on the remote, use

    git push origin l7

To choose either option permanently, see push.default in 'git help config'.


# [16.09]

In [3]:
gunzip sequences_2020-06-16_13-19.fasta.gz

In [6]:
makeblastdb -in sequences_2020-06-16_13-19.fasta -out uniprot_human -dbtype prot



Building a new DB, current time: 09/16/2020 16:03:22
New DB name:   /data11/bio/PROJECTS/CoV/commotSeqsWithHuman/uniprot_human
New DB title:  sequences_2020-06-16_13-19.fasta
Sequence type: Protein
Keep MBits: T
Maximum file size: 1000000000B
Adding sequences from FASTA; added 54715 sequences in 1.02822 seconds.


In [5]:
cp ../metatranscriptome/ref/wuhan_hu_1.gb .

In [2]:
#faa_sars="../gisaid/gisaid/prokka/hCoV_19_Wuhan_Hu_1/hCoV_19_Wuhan_Hu_1.faa"

In [8]:
GB2faa.pl wuhan_hu_1.gb > wuhan_hu_1.faa
faa_sars="wuhan_hu_1.faa"

In [15]:
blastp -db uniprot_human -query $faa_sars -outfmt 6 -evalue 30 > blast_results
wc -l blast_results

143 blast_results


In [10]:
blastp -db uniprot_human -query $faa_sars > blast_results_full

### look blast results
***

In [4]:
blast = read.delim("blast_results", head = F)

In [6]:
colnames(blast) = c("qid", "sid", "identity", "length", "mismatches", "gaps", "qstart", "qend", "sstart", "send", "evalue", "bitscore")

In [10]:
blast = blast[order(-blast$identity),]


In [11]:
head(blast)

,qid,sid,identity,length,mismatches,gaps,qstart,qend,sstart,send,evalue,bitscore
14,hCoV_19_Wuhan_Hu_1_00002,tr|A0A5K1VW56|A0A5K1VW56_HUMAN,53.846,26,12,0,1096,1121,237,262,3.2,32.7
44,hCoV_19_Wuhan_Hu_1_00004,tr|A0A0A0MT16|A0A0A0MT16_HUMAN,50.000,20,10,0,127,146,12,31,3.4,29.6
50,hCoV_19_Wuhan_Hu_1_00007,tr|K7EQ58|K7EQ58_HUMAN,47.368,19,10,0,35,53,116,134,3.7,26.6
51,hCoV_19_Wuhan_Hu_1_00007,tr|E9PL27|E9PL27_HUMAN,47.368,19,10,0,35,53,116,134,3.8,26.6
52,hCoV_19_Wuhan_Hu_1_00007,tr|E9PRR6|E9PRR6_HUMAN,47.368,19,10,0,35,53,116,134,4.5,26.6
57,hCoV_19_Wuhan_Hu_1_00008,tr|B4DWB8|B4DWB8_HUMAN,44.828,29,15,1,76,103,140,168,2.3,27.7
